# Pipeline 2
+ knowledge base
+ vector database
+ rag

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install python-dotenv

In [ ]:
from dotenv import load_dotenv
load_dotenv("/content/drive/Shareddrives/Knowledge Graph/code/pipeline2/.env", override=True)

True

## Knowledge base
In this component we perform the loading of the knowledge base

In [ ]:
%cd /content/drive/Shareddrives/Knowledge Graph/code/pipeline2/0.knowledge_base
!pip install -r requirements.txt

/content/drive/Shareddrives/Knowledge Graph/code/pipeline2/0.knowledge_base


In [ ]:
%cd /content/drive/Shareddrives/Knowledge Graph/code/pipeline2/files

/content/drive/Shareddrives/Knowledge Graph/code/pipeline2/files


In [ ]:
import rdflib
import json
import os

def knowledge_base():
  def parse_rdf(file_path):
      g = rdflib.Graph()
      try:
          g.parse(file_path, format='ttl')
      except rdflib.exceptions.ParserError as e:
          print(f"Errore di parsing: {e}")
      triples = [(str(s), str(p), str(o)) for s, p, o in g]
      return triples

  input_path = os.getenv("input_path")
  rdf_triples = parse_rdf(input_path)
  print(f'{len(rdf_triples)} triples extracted')
  with open(os.getenv("output_path"), 'w', encoding='utf-8') as f:
    json.dump(rdf_triples, f, ensure_ascii=False, indent=4)
if __name__ == '__main__':
  knowledge_base()

31310 triples extracted


In [ ]:
with open("/content/triples.json", "r") as file:
      triples = json.load(file)

In [ ]:
for triple in triples:
  print(triple)

['http://example.org/entities/MailDatabase', 'http://www.w3.org/1999/02/22-rdf-syntax-ns#type', 'http://example.org/d3f/Database']
['http://example.org/entities/IMAPServer', 'http://example.org/d3f/reads', 'http://example.org/entities/MailDatabase']
['http://example.org/entities/EmailClient', 'http://example.org/d3f/SendsEmail', 'http://example.org/entities/SMTPServer']
['http://example.org/entities/EmailClient', 'http://www.w3.org/2000/01/rdf-schema#label', 'Email Client']
['http://example.org/entities/IMAPServer', 'http://www.w3.org/2000/01/rdf-schema#label', 'IMAP Server']
['http://example.org/entities/MailDatabase', 'http://www.w3.org/2000/01/rdf-schema#label', 'Mail Database']
['http://example.org/entities/SMTPServer', 'http://example.org/d3f/writes', 'http://example.org/entities/MailDatabase']
['http://example.org/entities/SMTPServer', 'http://www.w3.org/1999/02/22-rdf-syntax-ns#type', 'http://example.org/d3f/MailServer']
['http://example.org/entities/IMAPServer', 'http://example

## Vector Database

In [ ]:
%cd /content/drive/Shareddrives/Knowledge Graph/code/pipeline2/1.vector_database
!pip install -r requirements.txt

/content/drive/Shareddrives/Knowledge Graph/code/pipeline2/1.vector_database
  Cloning https://github.com/pykeen/pykeen.git to /tmp/pip-req-build-p3vfk5no
  Running command git clone --filter=blob:none --quiet https://github.com/pykeen/pykeen.git /tmp/pip-req-build-p3vfk5no
  Resolved https://github.com/pykeen/pykeen.git to commit 0eb14895ebc5d803f268fa23a0b5504a255ca3a9
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 330.4/330.4 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.8/223.8 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 53.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 45.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 364.4/364.4 kB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━

In [ ]:
%cd /content/drive/Shareddrives/Knowledge Graph/code/pipeline2/files

/content/drive/Shareddrives/Knowledge Graph/code/pipeline2/files


In [ ]:
import weaviate
from weaviate.classes.init import Auth
from weaviate.classes.config import Configure, Property, DataType
import re
import os
import json

def extract_name(url):
    prefixes = [
        "http://d3fend.mitre.org/ontologies/d3fend.owl#",
        "http://www.w3.org/2000/01/rdf-schema#",
        "http://example.org/entities/",
        "http://example.org/d3f/",
        "http://www.w3.org/1999/02/22-rdf-syntax-ns#",
        "http://www.w3.org/2002/07/owl#",
        "http://www.w3.org/2004/02/skos/core#"
    ]
    for prefix in prefixes:
        if url.startswith(prefix):
            return re.split(r'[#/]', url)[-1]
    return url

def vector_database():
    weaviate_url = "https://s2w2oo8lrzmjuiril0nfjg.c0.europe-west3.gcp.weaviate.cloud"
    weaviate_api_key = "GbdiUUKHlYa2QI53GTckGxPfhNZFaslPcVs1"

    client = weaviate.connect_to_weaviate_cloud(
        cluster_url=weaviate_url,
        auth_credentials=Auth.api_key(weaviate_api_key),
        headers={
            "X-OpenAI-Api-Key": os.environ["OPENAI_API_TOKEN"]
        }
    )

    print(client.is_ready())

    client.collections.create(
        "Triple",
        vectorizer_config=Configure.Vectorizer.text2vec_openai(),
        properties=[
            Property(name="triple", data_type=DataType.TEXT),
            Property(name="source", data_type=DataType.TEXT)
        ]
    )

    with open(os.getenv("output_path"), "r") as file:
        triples = json.load(file)

    for index, triple in enumerate(triples, start=1):
        subject = extract_name(triple[0])
        predicate = extract_name(triple[1])
        object_ = extract_name(triple[2])
        source = subject + '--' + predicate + '--' + object_

        # Payload da caricare
        data_object = {
            "triple": source,
            "source": source
        }

        collection = client.collections.get("Triple")

        with collection.batch.dynamic() as batch:
            batch.add_object(properties=data_object)

        # Stampa lo stato di avanzamento
        print(f"Caricata tripla {index} di {len(triples)}: {data_object}")

if __name__ == '__main__':
    vector_database()


True
Caricata tripla 1 di 31310: {'triple': 'Dyna-Q--label--Dyna-Q', 'source': 'Dyna-Q--label--Dyna-Q'}
Caricata tripla 2 di 31310: {'triple': 'T1555.005--definition--Adversaries may acquire user credentials from third-party password managers.(Citation: ise Password Manager February 2019) Password managers are applications designed to store user credentials, normally in an encrypted database. Credentials are typically accessible after a user provides a master password that unlocks the database. After the database is unlocked, these credentials may be copied to memory. These databases can be stored as files on disk.(Citation: ise Password Manager February 2019)', 'source': 'T1555.005--definition--Adversaries may acquire user credentials from third-party password managers.(Citation: ise Password Manager February 2019) Password managers are applications designed to store user credentials, normally in an encrypted database. Credentials are typically accessible after a user provides a maste

ERROR:weaviate-client:{'message': 'Failed to send all objects in a batch of 1', 'error': "WeaviateInsertManyAllFailedError('Every object failed during insertion. Here is the set of all errors: connection to: OpenAI API failed with status: 503 request-id: req_09894545b7fdc937f383adfe09626339 error: The engine is currently overloaded. Please try again later.')"}
ERROR:weaviate-client:{'message': 'Failed to send 1 objects in a batch of 1. Please inspect client.batch.failed_objects or collection.batch.failed_objects for the failed objects.'}


Caricata tripla 245 di 31310: {'triple': 'Vulnerability--subClassOf--D3FENDCore', 'source': 'Vulnerability--subClassOf--D3FENDCore'}
Caricata tripla 246 di 31310: {'triple': 'T1105--subClassOf--CommandAndControlTechnique', 'source': 'T1105--subClassOf--CommandAndControlTechnique'}
Caricata tripla 247 di 31310: {'triple': 'UserBehaviorAnalysis--type--NamedIndividual', 'source': 'UserBehaviorAnalysis--type--NamedIndividual'}
Caricata tripla 248 di 31310: {'triple': 'HTTPURL--type--NamedIndividual', 'source': 'HTTPURL--type--NamedIndividual'}
Caricata tripla 249 di 31310: {'triple': 'KioskComputer--label--Kiosk Computer', 'source': 'KioskComputer--label--Kiosk Computer'}
Caricata tripla 250 di 31310: {'triple': 'LinuxOpenAtArgumentO_CREAT--label--Linux OpenAt Argument O_CREAT', 'source': 'LinuxOpenAtArgumentO_CREAT--label--Linux OpenAt Argument O_CREAT'}
Caricata tripla 251 di 31310: {'triple': 'CWE-103--label--Struts: Incomplete validate() Method Definition', 'source': 'CWE-103--label--S

/usr/local/lib/python3.10/dist-packages/weaviate/warnings.py:280: UserWarning: Bat003: The dynamic batch-size could not be refreshed successfully: error UnexpectedStatusCodeError('Nodes status! Unexpected status code: 503, with response body: None.')
  warnings.warn(


Caricata tripla 1268 di 31310: {'triple': 'T1547--label--Boot or Logon Autostart Execution', 'source': 'T1547--label--Boot or Logon Autostart Execution'}
Caricata tripla 1269 di 31310: {'triple': 'ProcessSpawnAnalysis--kb-reference--Reference-CAR-2020-04-001%3AShadowCopyDeletion_MITRE', 'source': 'ProcessSpawnAnalysis--kb-reference--Reference-CAR-2020-04-001%3AShadowCopyDeletion_MITRE'}
Caricata tripla 1270 di 31310: {'triple': 'Reference-TivoliApplicationDependencyDiscoverManager7_3_0DependenciesBetweenResources--kb-reference-title--Tivoli Application Dependency Discovery Manager 7.3.0 - Dependencies between resources', 'source': 'Reference-TivoliApplicationDependencyDiscoverManager7_3_0DependenciesBetweenResources--kb-reference-title--Tivoli Application Dependency Discovery Manager 7.3.0 - Dependencies between resources'}
Caricata tripla 1271 di 31310: {'triple': 'SenderMTAReputationAnalysis--type--SenderMTAReputationAnalysis', 'source': 'SenderMTAReputationAnalysis--type--SenderMTAR

...

Caricata tripla 3370 di 31310: {'triple': 'n38e1d26c99874373b85210c7b3375474b450--type--Restriction', 'source': 'n38e1d26c99874373b85210c7b3375474b450--type--Restriction'}
Caricata tripla 3371 di 31310: {'triple': 'CCI-000195_v2022-04-05--type--CCIControl', 'source': 'CCI-000195_v2022-04-05--type--CCIControl'}
Caricata tripla 3372 di 31310: {'triple': 'CCI-002397_v2022-04-05--narrower--InboundTrafficFiltering', 'source': 'CCI-002397_v2022-04-05--narrower--InboundTrafficFiltering'}
Caricata tripla 3373 di 31310: {'triple': 'CWE-707--subClassOf--Weakness', 'source': 'CWE-707--subClassOf--Weakness'}
Caricata tripla 3374 di 31310: {'triple': 'T1498.002--definition--Adversaries may attempt to cause a denial of service (DoS) by reflecting a high-volume of network traffic to a target. This type of Network DoS takes advantage of a third-party server intermediary that hosts and will respond to a given spoofed source IP address. This third-party server is commonly termed a reflector. An adversar

ERROR:weaviate-client:{'message': 'Failed to send all objects in a batch of 1', 'error': "WeaviateInsertManyAllFailedError('Every object failed during insertion. Here is the set of all errors: connection to: OpenAI API failed with status: 503 request-id: req_70d7c45ca452167963b9bdf2a2893356 error: The engine is currently overloaded. Please try again later.')"}
ERROR:weaviate-client:{'message': 'Failed to send 1 objects in a batch of 1. Please inspect client.batch.failed_objects or collection.batch.failed_objects for the failed objects.'}


Caricata tripla 3870 di 31310: {'triple': 'n38e1d26c99874373b85210c7b3375474b927--type--Restriction', 'source': 'n38e1d26c99874373b85210c7b3375474b927--type--Restriction'}
Caricata tripla 3871 di 31310: {'triple': 'T1553.001--type--NamedIndividual', 'source': 'T1553.001--type--NamedIndividual'}
Caricata tripla 3872 di 31310: {'triple': 'T1070.005--subClassOf--n38e1d26c99874373b85210c7b3375474b159', 'source': 'T1070.005--subClassOf--n38e1d26c99874373b85210c7b3375474b159'}
Caricata tripla 3873 di 31310: {'triple': 'PageTable--type--Class', 'source': 'PageTable--type--Class'}
Caricata tripla 3874 di 31310: {'triple': 'CCI-001199_v2022-04-05--label--CCI-001199', 'source': 'CCI-001199_v2022-04-05--label--CCI-001199'}
Caricata tripla 3875 di 31310: {'triple': 'CCI-002690_v2022-04-05--published--2013-07-11T00:00:00', 'source': 'CCI-002690_v2022-04-05--published--2013-07-11T00:00:00'}
Caricata tripla 3876 di 31310: {'triple': 'CCI-001991_v2022-04-05--type--CCIControl', 'source': 'CCI-001991_v2

ERROR:weaviate-client:{'message': 'Failed to send all objects in a batch of 1', 'error': 'WeaviateBatchError(\'Query call with protocol GRPC batch failed with message <AioRpcError of RPC that terminated with:\\n\\tstatus = StatusCode.UNAVAILABLE\\n\\tdetails = "Received http2 header with status: 502"\\n\\tdebug_error_string = "UNKNOWN:Error received from peer  {created_time:"2025-01-15T22:55:28.25709891+00:00", grpc_status:14, grpc_message:"Received http2 header with status: 502"}"\\n>.\')'}
ERROR:weaviate-client:{'message': 'Failed to send 1 objects in a batch of 1. Please inspect client.batch.failed_objects or collection.batch.failed_objects for the failed objects.'}


Caricata tripla 5176 di 31310: {'triple': 'T1187--may-modify--WindowsShortcutFile', 'source': 'T1187--may-modify--WindowsShortcutFile'}
Caricata tripla 5177 di 31310: {'triple': 'n38e1d26c99874373b85210c7b3375474b1133--type--Restriction', 'source': 'n38e1d26c99874373b85210c7b3375474b1133--type--Restriction'}
Caricata tripla 5178 di 31310: {'triple': 'CCI-001499_v2022-04-05--narrower--UserAccountPermissions', 'source': 'CCI-001499_v2022-04-05--narrower--UserAccountPermissions'}
Caricata tripla 5179 di 31310: {'triple': 'CCICatalog_v2022-04-05--has-member--CCI-001453_v2022-04-05', 'source': 'CCICatalog_v2022-04-05--has-member--CCI-001453_v2022-04-05'}
Caricata tripla 5180 di 31310: {'triple': 'Reference-FileIntegrityMonitoringinMicrosoftDefenderforCloud-Microsoft--label--Reference - File Integrity Monitoring in Microsoft Defender for Cloud - Microsoft', 'source': 'Reference-FileIntegrityMonitoringinMicrosoftDefenderforCloud-Microsoft--label--Reference - File Integrity Monitoring in Micro

## RAG

In [ ]:
%cd /content/drive/Shareddrives/Knowledge Graph/code/pipeline2/2.rag
!cat requirements.txt

/content/drive/Shareddrives/Knowledge Graph/code/pipeline2/2.rag
langchain
langchain-community
weaviate-client
langchain-openai
langchain-weaviate

In [ ]:
%cd /content/drive/Shareddrives/Knowledge Graph/code/pipeline2/2.rag
!pip install -r requirements.txt

/content/drive/Shareddrives/Knowledge Graph/code/pipeline2/2.rag
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 28.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 37.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.4/325.4 kB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.8/223.8 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 57.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 52.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 27.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 454.8/454.8 kB 29.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.1/519.1 kB 30.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 51.2 MB/s e

In [ ]:
%cd /content/drive/Shareddrives/Knowledge Graph/code/pipeline2/files

/content/drive/Shareddrives/Knowledge Graph/code/pipeline2/files


In [ ]:
import os
from weaviate.classes.query import MetadataQuery
from langchain_openai import ChatOpenAI
import weaviate
from weaviate.classes.init import Auth
import textwrap

def rag():
    weaviate_url = os.getenv("WEAVIATE_URL")
    weaviate_api_key = os.getenv("WEAVIATE_API_KEY")

    client = weaviate.connect_to_weaviate_cloud(
        cluster_url=weaviate_url,
        auth_credentials=Auth.api_key(weaviate_api_key),
        headers={
            "X-OpenAI-Api-Key": os.environ["OPENAI_API_TOKEN"]
        }
    )

    print(client.is_ready())


    # Esegui ricerca di similarità
    def similarity_search(question: str):
        jeopardy = client.collections.get("Triple")
        response = jeopardy.query.near_text(
            query=question,
            limit=4,
            return_metadata=MetadataQuery(distance=True)
        )
        return response.objects

    # Funzione per generare una risposta dal modello LLM
    def generate_RAG_answer(question: str, context: str):
        llm = ChatOpenAI(
            temperature=0,
            api_key=os.getenv("OPENAI_API_TOKEN"),
            model_name="gpt-3.5-turbo"
        )
        prompt = f"Answer the question based on the context: \n\nContext: {context}\n\nQuestion: {question}"
        response = llm.invoke(prompt)
        return response.content

        # Funzione per generare una risposta dal modello LLM
    def generate_LLM_answer(question: str):
        llm = ChatOpenAI(
            temperature=0,
            api_key=os.getenv("OPENAI_API_TOKEN"),
            model_name="gpt-3.5-turbo"
        )
        response = llm.invoke(question)
        return response.content

    # Funzione per stampare i risultati in un formato leggibile
    def pretty_print_response(question: str):
        print(f"Question: {question}")
        print("=" * 90)
        print("Similarity Search Results:")
        # Recupera gli oggetti simili
        context = similarity_search(question)
        for o in context:
          print(o.properties)
          print(o.metadata.distance)
        print("=" * 90)
        # Genera la risposta basata sul contesto
        answer = generate_RAG_answer(question, context)
        print("=" * 90)
        print("RAG Answer:")
        print(textwrap.fill(answer, 60))
        print("=" * 90)
        print("LLM Answer:")
        print(textwrap.fill(generate_LLM_answer(question), 60))
        print("=" * 90)

    question = "What does the T1115 technique read?"
    pretty_print_response(question)

    print("")
    print("")
    question2 = "What is the definition of PacketLog?"
    pretty_print_response(question2)

    print("")
    print("")
    question3 = "What offensive techniques may modify Windows Registry Key?"
    pretty_print_response(question3)

    print("")
    print("")
    question4 = "What techniques may access the Mail Server?"
    pretty_print_response(question4)

    print("")
    print("")
    question5 = "What is the definition of EmailRemoval?"
    pretty_print_response(question5)




if __name__ == "__main__":
    rag()

True
Question: What does the T1115 technique read?
Similarity Search Results:
{'triple': 'T1114.001--reads--Email', 'source': 'T1114.001--reads--Email'}
0.39078670740127563
{'triple': 'T1115--reads--Clipboard', 'source': 'T1115--reads--Clipboard'}
0.4021607041358948
{'triple': 'T1100--comment--This technique has been revoked by T1505.003', 'source': 'T1100--comment--This technique has been revoked by T1505.003'}
0.42233288288116455
{'triple': 'T1150--comment--This technique has been revoked by T1547.011', 'source': 'T1150--comment--This technique has been revoked by T1547.011'}
0.43406248092651367
RAG Answer:
The T1115 technique reads Clipboard.
LLM Answer:
The T1115 technique is a Windows registry modification
technique that involves reading the Windows registry to
gather information about the system and its configuration.
This technique is commonly used by threat actors to gather
information for reconnaissance purposes.


Question: What is the definition of PacketLog?
Similarity Sear

In [ ]:
import os
from weaviate.classes.query import MetadataQuery
from langchain_openai import ChatOpenAI
import weaviate
from weaviate.classes.init import Auth
import textwrap

def rag():
    weaviate_url = os.getenv('WEAVIATE_URL_ARCH')
    weaviate_api_key = os.getenv('WEAVIATE_API_KEY_ARCH')

    client = weaviate.connect_to_weaviate_cloud(
        cluster_url=weaviate_url,
        auth_credentials=Auth.api_key(weaviate_api_key),
        headers={
            "X-OpenAI-Api-Key": os.environ["OPENAI_API_TOKEN"]
        }
    )

    print(client.is_ready())


    # Esegui ricerca di similarità
    def similarity_search(question: str):
        jeopardy = client.collections.get("Triple")
        response = jeopardy.query.near_text(
            query=question,
            limit=4,
            return_metadata=MetadataQuery(distance=True)
        )
        return response.objects

    # Funzione per generare una risposta dal modello LLM
    def generate_RAG_answer(question: str, context: str):
        llm = ChatOpenAI(
            temperature=0,
            api_key=os.getenv("OPENAI_API_TOKEN"),
            model_name="gpt-3.5-turbo"
        )
        prompt = f"Answer the question based on the context: \n\nContext: {context}\n\nQuestion: {question}"
        response = llm.invoke(prompt)
        return response.content

        # Funzione per generare una risposta dal modello LLM
    def generate_LLM_answer(question: str):
        llm = ChatOpenAI(
            temperature=0,
            api_key=os.getenv("OPENAI_API_TOKEN"),
            model_name="gpt-3.5-turbo"
        )
        response = llm.invoke(question)
        return response.content

    # Funzione per stampare i risultati in un formato leggibile
    def pretty_print_response(question: str):
        print(f"Question: {question}")
        print("=" * 90)
        print("Similarity Search Results:")
        # Recupera gli oggetti simili
        context = similarity_search(question)
        for o in context:
          print(o.properties)
          print(o.metadata.distance)
        print("=" * 90)
        # Genera la risposta basata sul contesto
        answer = generate_RAG_answer(question, context)
        print("=" * 90)
        print("RAG Answer:")
        print(textwrap.fill(answer, 60))
        print("=" * 90)
        print("LLM Answer:")
        print(textwrap.fill(generate_LLM_answer(question), 60))
        print("=" * 90)

    question = "What does the user do in my architecture?"
    pretty_print_response(question)
    print("--------------------------------------------------------------------------------------")
    question2 = "How many types of mail servers do I have in my architecture?"
    pretty_print_response(question2)
    print("--------------------------------------------------------------------------------------")
    question3 = "Which mail server retrieves emails for the user?"
    pretty_print_response(question3)


if __name__ == "__main__":
    rag()

True
Question: What does the user do in my architecture?
Similarity Search Results:
{'triple': 'EmailClient--type--User', 'source': 'EmailClient--type--User'}
0.7078037261962891
{'triple': 'IMAPServer--reads--MailDatabase', 'source': 'IMAPServer--reads--MailDatabase'}
0.7525630593299866
{'triple': 'IMAPServer--EmailRetrieval--EmailClient', 'source': 'IMAPServer--EmailRetrieval--EmailClient'}
0.7657593488693237
{'triple': 'SMTPServer--writes--MailDatabase', 'source': 'SMTPServer--writes--MailDatabase'}
0.7871342897415161
RAG Answer:
In your architecture, the user is associated with the
'EmailClient' type.
LLM Answer:
In your architecture, the user interacts with the system by
inputting commands, requests, or data through the user
interface. The user's actions trigger processes within the
system, such as data processing, communication with external
systems, or generating outputs. The user may also receive
feedback, notifications, or results from the system based on
their interactions. Ov